In [247]:
import implicit
import pandas
import time
from tqdm import tqdm
from sortedcontainers import SortedList, SortedDict
from sklearn.metrics import mean_squared_error

In [270]:
path = "~/Downloads/ml-1m/"
ratings = pandas.read_csv(f'{path}ratings.dat', sep = "::", names = ["userId", "movieId", "explicit", "timestamp"])
movies = pandas.read_csv(f'{path}movies.dat', sep="::", encoding = "ISO-8859-1", names = ["movieId", "title"], usecols = [0, 1])

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_81803/3920604715.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pandas.read_csv(f'{path}ratings.dat', sep = "::", names = ["userId", "movieId", "explicit", "timestamp"])
/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_81803/3920604715.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pandas.read_csv(f'{path}movies.dat', sep="::", encoding = "ISO-8859-1", names = ["movieId", "title"], usecols = [0, 1])


In [384]:
implicit_ratings = ratings.copy(deep = True)
implicit_ratings['rating'] = [1 if row.explicit > 3 else 0 for row in implicit_ratings.itertuples()]

def split_data(df):  
    start_ts = df['timestamp'].min()
    med_ts = df['timestamp'].quantile(.5)
    end_ts = df['timestamp'].max()
    train_df = df[df['timestamp'] <= med_ts]
    stream_df = df[df['timestamp'] > med_ts]
    seen_movies = set(train_df['movieId'])
    print(len(seen_movies), len(set(stream_df['movieId'])), len(stream_df))
    stream_df = stream_df.drop(stream_df[stream_df['movieId'].map(lambda x: x not in seen_movies)].index)
    print(len(set(stream_df['movieId'])), len(stream_df))
    '''
    seen_users = set(train_df['user_id'])
    num_stream_users = set(stream_df['user_id'])
    stream_df = stream_df.drop(stream_df[stream_df['user_id'].map(lambda x: x not in seen_users)].index)
    print(len(seen_users), len(num_stream_users), len(set(stream_df['user_id'])), len(stream_df))
    '''
    return train_df, stream_df

train_df, stream_df = split_data(implicit_ratings)

3551 3643 500104
3488 499218


In [185]:
train_df.max()

userId            6040
movieId           3952
rating               0
timestamp    973018006
explicit             0
dtype: int64

In [63]:
max_movie = train_df.max()[1]
if stream_df.max()[1] > max_movie:
    max_movie = stream_df.max()[1]
max_movie += 1
max_movie

3953

In [304]:
max_user = train_df.max()[0]
if stream_df.max()[0] > max_user:
    max_user = stream_df.max()[0]
max_user += 1
max_user

6041

In [385]:
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import h5py
output_filename = "/Users/amitnarang/new_experiments/hfd5_output"
m = coo_matrix(
    (train_df["rating"].astype(np.float32), (train_df["movieId"], train_df["userId"]))
).tocsr()

train_plays = csr_matrix((m.data, m.indices, m.indptr))
train_plays

<3953x6041 sparse matrix of type '<class 'numpy.float32'>'
	with 500105 stored elements in Compressed Sparse Row format>

In [356]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(train_plays)

  0%|          | 0/15 [00:00<?, ?it/s]

In [357]:
data_to_send = dict()
ts_list = stream_df['timestamp'].unique()
for ts in ts_list:
    curr_data = stream_df[stream_df['timestamp'] == ts]
    data_to_send[ts] = curr_data

In [326]:
def generate_liked_movies():
    liked_movies = dict()
    user_list = train_df['userId'].unique()
    for user_id in user_list:
        curr_data = train_df[train_df['userId'] == user_id]
        liked_data = curr_data[curr_data['rating'] == 1]
        liked_movies[user_id] = liked_data['movieId'].tolist()
    return liked_movies

In [239]:
class FIFOScheduler:
    
    def __init__(self):
        self.queue = []
        self.name = "fifo"
        
    def push(self, row):
        self.queue.append(row)
    
    def pop(self):
        if len(self.queue) == 0:
            return None
        return self.queue.pop()

In [378]:
class BaselineScheduler:
    
    def __init__(self):
        self.name = "baseline"
        
    def push(self, row):
        pass
    
    def pop(self):
        pass

In [250]:
class MaxPendingScheduler:
    
    def __init__(self):
        # items = [key, num_pending]
        self.updated_list = SortedList(key = lambda x: x[1])
        self.row_dict = dict()
        self.name = "pending"

    def push(self, row):
        user = row.userId
        if user not in self.row_dict:
            self.updated_list.add([user, 1])
            self.row_dict[user] = [row]
        else:
            pending_updates = len(self.row_dict[user])
            self.updated_list.remove([user, pending_updates])
            self.updated_list.add([user, pending_updates + 1])
            self.row_dict[user].append(row)
        
    def pop(self):
        if len(self.updated_list) > 0:
            user = self.updated_list[-1][0]
            num_pending = len(self.row_dict[user])
            row = self.row_dict[user].pop()
            if num_pending == 1:
                self.updated_list.pop(-1)
                del self.row_dict[user]
            else:
                self.updated_list.remove([user, num_pending])
                self.updated_list.add([user, num_pending - 1])
            return row

In [321]:
class FewestUpdateScheduler:
    
    def __init__(self):
        self.updated_list = SortedList(key = lambda x: x[1])
        self.row_dict = dict()
        self.name = "fewest"

    def push(self, row):
        user = row.userId
        if user not in self.row_dict:
            self.updated_list.add([user, 0])
            self.row_dict[user] = [row]
        else:
            self.row_dict[user].append(row)
          
    def pop(self):
        user_data = self.updated_list.pop(0)
        user = user_data[0]
        row = self.row_dict[user].pop()
        if len(self.row_dict[user]) != 0:
            user_data[1] = user_data[1] + 1
            self.updated_list.add(user_data)
        #self.updated_list[0][1] = self.updated_list[0][1] + 1
        return row

In [353]:
class ImplicitModelUpdater:
    def __init__(self, model):
        self.model = model
        self.name = "implicit"
        self.liked_movies = generate_liked_movies()
        
    def update(self, row):
            rows = []
            cols = []
            data = []
                
            movie = row.movieId
            user = row.userId
            rating = row.rating
            if user not in self.liked_movies:
                self.liked_movies[user] = []
            if rating == 1:
                self.liked_movies[user].append(movie)
            rows.append(0)
            cols.append(movie)
            data.append(rating)
            csr_shape = (1, max_movie)
            user_items = csr_matrix((data, (rows, cols)), shape=csr_shape)
            curr = time.time()
            self.model.partial_fit_users([user], user_items)
            #print(time.time() - curr)
        
    def predict(self, row):
        user = row.userId
        movie = row.movieId
        cols = []
        if user in self.liked_movies:
            cols = self.liked_movies[user]
        rows = [user] * len(cols)
        data = [1] * len(cols)
        user_items = coo_matrix((data, (rows, cols)), shape=(max_user, max_movie))
        #print(user_items)
        explanation = self.model.explain(user, user_items, movie)
        prediction = explanation[0]
        return prediction

In [367]:
def simulate(scheduler, updater, stream, df, update_cadence, num_update):
    start_ts = df['timestamp'].min()
    end_ts = df['timestamp'].max()
    
    predictions = []
    next_feature_update_time = start_ts + update_cadence
    num_updates = 0
    num_processed = 0
    
    stream_ts = sorted(list(stream.keys()), key = lambda x: int(x))
    
    for ts in tqdm(stream_ts):
        curr_data = stream[ts]
        for row in curr_data.itertuples():
            scheduler.push(row)
            predictions.append(updater.predict(row))
        num_processed += len(curr_data)
        if ts >= next_feature_update_time:
            # how many records to process
            row = scheduler.pop()
            if row:
                updater.update(row)
                num_updates += 1
            next_feature_update_time += update_cadence
    return predictions

In [387]:
def run_simulations(scheduler_list, updater_list, stream, df, update_cadences, num_updates):
    experiments = []
    for scheduler in scheduler_list:
        print(f'Scheduler: {scheduler.name}')
        for update_cadence in update_cadences:
            for num_update in num_updates:
                model = implicit.als.AlternatingLeastSquares(factors=50)
                model.fit(train_plays)
                updater = ImplicitModelUpdater(model)
                experiment_name = f'{scheduler.name}_{updater.name}_{update_cadence}_{num_update}_predictions'
                start = time.time()
                predictions = simulate(scheduler, updater, stream, df, update_cadence, num_update)
                elapsed = time.time() - start
                df[experiment_name] = predictions
                df[scheduler.name] = round(df[experiment_name])
                mse = mean_squared_error(df['rating'], df[experiment_name])
                mse_round = mean_squared_error(df['rating'], df[scheduler.name])
                experiments.append([experiment_name, elapsed, mse, mse_round])
    return df, experiments

In [388]:
schedulers = [BaselineScheduler(), FIFOScheduler(), MaxPendingScheduler(), FewestUpdateScheduler()]
updaters = [ImplicitModelUpdater(model)]
stream, experiments = run_simulations(schedulers, updaters, data_to_send, stream_df, [1000], [10])

Scheduler: baseline


  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████| 230742/230742 [02:28<00:00, 1553.45it/s]


Scheduler: fifo


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████| 230742/230742 [02:58<00:00, 1295.23it/s]


Scheduler: fewest


  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████| 230742/230742 [02:46<00:00, 1388.63it/s]


In [389]:
stream

,userId,movieId,explicit,timestamp,rating,baseline_implicit_1000_10_predictions,baseline,fifo_implicit_1000_10_predictions,fifo,pending_implicit_1000_10_predictions,pending,fewest_implicit_1000_10_predictions,fewest
0,1,1193,5,978300760,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,1,661,3,978302109,0,-0.005366,-0.0,-0.004371,-0.0,-0.007758,-0.0,-0.009116,-0.0
2,1,914,3,978301968,0,0.009411,0.0,0.009065,0.0,0.005993,0.0,0.003378,0.0
3,1,3408,4,978300275,1,0.007415,0.0,0.009223,0.0,0.004479,0.0,0.009504,0.0
4,1,2355,5,978824291,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000126,6040,1333,4,997454140,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1000162,6040,3362,4,997453796,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1000167,6040,161,3,997454486,0,0.004570,0.0,0.005231,0.0,0.003242,0.0,0.003793,0.0
1000169,6040,2725,4,997454180,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [394]:
max(stream['baseline'])

0.0

In [390]:
baseline = mean_squared_error(stream['rating'], stream['baseline'])
fifo_mse = mean_squared_error(stream['rating'], stream['fifo'])
pending_mse = mean_squared_error(stream['rating'], stream['pending'])
fewest_mse = mean_squared_error(stream['rating'], stream['fewest'])
baseline, fifo_mse, pending_mse, fewest_mse

(0.5687174741295385,
 0.5687174741295385,
 0.5687174741295385,
 0.5687174741295385)

In [377]:
experiments

[['fifo_implicit_1000_10_predictions',
  186.3884220123291,
  0.5682277550621297,
  0.5687174741295385],
 ['pending_implicit_1000_10_predictions',
  182.12898111343384,
  0.5683077844094129,
  0.5687174741295385],
 ['fewest_implicit_1000_10_predictions',
  173.6549949645996,
  0.5683425110691159,
  0.5687174741295385]]

In [294]:
set([type(x) for x in list(data_to_send.keys())])

{numpy.int64}

In [272]:
stream_df

,userId,movieId,explicit,timestamp,rating
0,1,1193,5,978300760,1
1,1,661,3,978302109,0
2,1,914,3,978301968,0
3,1,3408,4,978300275,1
4,1,2355,5,978824291,1
...,...,...,...,...,...
1000126,6040,1333,4,997454140,1
1000162,6040,3362,4,997453796,1
1000167,6040,161,3,997454486,0
1000169,6040,2725,4,997454180,1
